In [9]:
# importing libraries
import numpy as np
import pandas as pd
import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
  #  for filename in filenames:
       # print(os.path.join(dirname, filename))

In [10]:
# Libraries for pre-processing and evaluation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# Training model with kfold strategy
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import TimeSeriesSplit

## Data loading

In [11]:
train_identity = pd.read_csv('ieee-fraud-detection/train_identity.csv')
train_transaction = pd.read_csv('ieee-fraud-detection/train_transaction.csv')

In [12]:
test_identity = pd.read_csv('ieee-fraud-detection/test_identity.csv')
test_transaction = pd.read_csv('ieee-fraud-detection/test_transaction.csv')

In [13]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

## Feature engineering

# Merging transaction data with identity data

In [14]:
train = pd.merge(train_transaction,train_identity,on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

In [15]:
print('train dataset shape', train.shape)
print('test dataset shape', test.shape)

train dataset shape (590540, 434)
test dataset shape (506691, 433)


In [16]:
del train_identity, train_transaction, test_identity, test_transaction

In [17]:
# solving incosistent naming of columns

for i in range(1,39):
    if i < 10:
          test.rename(columns = {"id-0"+str(i) : "id_0"+str(i)}, inplace=True)
    else:
          test.rename(columns = {"id-"+str(i) : "id_"+str(i)}, inplace = True)

### Droping columns having null values greater than 90%

In [18]:
cols_lotof_nulls = [c for c in train.columns if (train[c].isnull().sum() / train.shape[0])>0.90]
cols_lotof_nulls_test = [c for c in test.columns if (test[c].isnull().sum() / test.shape[0])>0.90]


cols_to_drop = list(set(cols_lotof_nulls+ cols_lotof_nulls_test))
len(cols_to_drop)

train = train.drop(cols_to_drop, axis=1)
test = test.drop(cols_to_drop, axis=1)

In [19]:
# taking categorical colums to give as paratmeter to TabNet classifier
cat_cols = list(train.select_dtypes(['object']).columns)

len(cat_cols)



29

## Encoding categorical variables

In [20]:
# Using Label encoder to convert categorical to numerical
for col in cat_cols:
  if col in train.columns:
    le = LabelEncoder()
    le.fit(list(train[col].astype(str).values) + list(test[col].astype(str).values))
    train[col] = le.transform(list(train[col].astype('str').values))
    test[col] = le.transform(list(test[col].astype('str').values))

In [21]:
# Replacing inf values if any
train = train.replace([np.inf, -np.inf], np.nan)
test = test.replace([np.inf, -np.inf], np.nan)

In [22]:
# Removing columns if they dont have atleast 100000 non-Nan values
train = train.dropna(axis=1, thresh = 250000)


In [23]:
# Removing Unnecessary columsn for traing data
X = train.drop(['isFraud', 'TransactionID'], axis=1)
y = train.isFraud

# For saving memory
del train

In [24]:
# Custom Loss Function
def log_loss_score(actual, predicted,  eps=1e-15):
    p1 = actual * np.log(predicted+eps)
    p0 = (1-actual) * np.log(1-predicted+eps)
    loss = p0 + p1

    return -loss.mean()

In [25]:
# Filling Nan values with -1
X = X.fillna(-1)

In [26]:
# Training model with kfold strategy
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import TimeSeriesSplit
strategy = "KFOLD"
num_ensembling = 1
device = 'cuda'
EPOCHS = 5
SPLITS = 5
save_name = 'tabnet'
if strategy == "KFOLD":
    oof_preds_all = []
    oof_targets_all = []
    scores_all =  []
    scores_auc_all= []
    for seed in range(num_ensembling):
        print("## SEED : ", seed)
        skf = TimeSeriesSplit(n_splits=SPLITS)
        oof_preds = []
        oof_targets = []
        scores = []
        scores_auc = []
        for j, (train_idx, val_idx) in enumerate(skf.split(X, y)):
            print("FOLDS : ", j)

            X_train = X.iloc[train_idx].values
            y_train = y[train_idx].values
            X_val, y_val = X.iloc[val_idx].values, y[val_idx].values
            model = TabNetClassifier(n_d=8, n_a=8, n_steps=1, gamma=1.3,
                                     lambda_sparse=0,optimizer_fn=torch.optim.Adam,
                                   optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                                     mask_type='entmax', device_name=device, output_dim=1,
                                     scheduler_params=dict(milestones=[100,150], gamma=0.9), 
                                     scheduler_fn=torch.optim.lr_scheduler.MultiStepLR)
            #'sparsemax'
            
            model.fit(X_train=X_train, y_train=y_train,  eval_set=[(X_train, y_train), (X_val, y_val)],max_epochs=EPOCHS,
                      patience=20, batch_size=1024, virtual_batch_size=128, eval_name=['train', 'valid'],)

            preds = model.predict(X_val)
            score = log_loss_score(y_val, preds)
            name = save_name + f"_fold{j}_{seed}"
            model.save_model(name)
            ## save oof to compute the CV later
            oof_preds.append(preds)
            oof_targets.append(y_val)
            scores.append(score)
            roc_ = roc_auc_score(y_val,preds)
            scores_auc.append(roc_)
            print(f"validation fold {j} : {score}, roc AUC Score: {roc_}")
        oof_preds_all.append(np.concatenate(oof_preds))
        
        oof_targets_all.append(np.concatenate(oof_targets))
        scores_all.append(np.array(scores))
        scores_auc_all.append(np.array(scores_auc))

## SEED :  0
FOLDS :  0
epoch 0  | loss: 0.13938 | train_auc: 0.75592 | valid_auc: 0.76374 |  0:00:12s
epoch 1  | loss: 0.1012  | train_auc: 0.79041 | valid_auc: 0.80027 |  0:00:24s
epoch 2  | loss: 0.09606 | train_auc: 0.83253 | valid_auc: 0.80236 |  0:00:36s
epoch 3  | loss: 0.0919  | train_auc: 0.85962 | valid_auc: 0.79806 |  0:00:48s
epoch 4  | loss: 0.08893 | train_auc: 0.87118 | valid_auc: 0.81481 |  0:00:59s
Stop training because you reached max_epochs = 5 with best_epoch = 4 and best_valid_auc = 0.81481


C:\Users\anant\Wells Fargo\ML Research Group\2023\Codes\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_fold0_0.zip
validation fold 0 : 1.0296045633913604, roc AUC Score: 0.5708433566383055
FOLDS :  1


C:\Users\anant\Wells Fargo\ML Research Group\2023\Codes\pytorch_tabnet\abstract_model.py:80: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.12903 | train_auc: 0.82183 | valid_auc: 0.82092 |  0:00:21s
epoch 1  | loss: 0.10363 | train_auc: 0.85856 | valid_auc: 0.83447 |  0:00:42s
epoch 2  | loss: 0.09863 | train_auc: 0.86628 | valid_auc: 0.8296  |  0:01:03s
epoch 3  | loss: 0.09595 | train_auc: 0.87149 | valid_auc: 0.82421 |  0:01:23s
epoch 4  | loss: 0.09457 | train_auc: 0.8763  | valid_auc: 0.81804 |  0:01:44s
Stop training because you reached max_epochs = 5 with best_epoch = 1 and best_valid_auc = 0.83447


...

Successfully saved model at tabnet_fold3_0.zip
validation fold 3 : 1.1036490633489533, roc AUC Score: 0.6394836936440157
FOLDS :  4


C:\Users\anant\Wells Fargo\ML Research Group\2023\Codes\pytorch_tabnet\abstract_model.py:80: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.12847 | train_auc: 0.83343 | valid_auc: 0.81673 |  0:00:45s
epoch 1  | loss: 0.11207 | train_auc: 0.8563  | valid_auc: 0.82351 |  0:01:29s
epoch 2  | loss: 0.10814 | train_auc: 0.86101 | valid_auc: 0.8495  |  0:02:15s
epoch 3  | loss: 0.10473 | train_auc: 0.85466 | valid_auc: 0.84302 |  0:03:03s
epoch 4  | loss: 0.10266 | train_auc: 0.82889 | valid_auc: 0.82883 |  0:03:50s
Stop training because you reached max_epochs = 5 with best_epoch = 2 and best_valid_auc = 0.8495


C:\Users\anant\Wells Fargo\ML Research Group\2023\Codes\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_fold4_0.zip
validation fold 4 : 1.0846992861086213, roc AUC Score: 0.5989719926116498


In [27]:
# Prediction on test set
X_test = test[X.columns].fillna(-1)
X_test = torch.tensor(X_test.values)
preds = model.predict_proba(X_test)

sub = pd.DataFrame({'TransactionID': test['TransactionID'].values.tolist(),
                    'isFraud': preds[:,1].tolist()
                   })

In [28]:
model.feature_importances_

array([0.00000000e+00, 0.00000000e+00, 1.95587054e-07, 0.00000000e+00,
       1.07638695e-02, 1.05028520e-06, 0.00000000e+00, 5.22111197e-07,
       4.26645772e-07, 3.16344430e-06, 1.77654590e-07, 3.21108560e-06,
       3.04825612e-08, 1.71587290e-04, 5.26687065e-02, 0.00000000e+00,
       0.00000000e+00, 4.42600410e-05, 1.15665820e-05, 0.00000000e+00,
       3.44831423e-03, 0.00000000e+00, 0.00000000e+00, 1.28147277e-05,
       0.00000000e+00, 0.00000000e+00, 2.28635172e-06, 0.00000000e+00,
       8.07334575e-03, 2.76681241e-08, 0.00000000e+00, 4.81156420e-06,
       0.00000000e+00, 8.30993791e-07, 1.72268855e-10, 1.33459261e-04,
       6.27422436e-03, 1.04510812e-05, 8.30315984e-08, 0.00000000e+00,
       5.26320089e-04, 8.85879947e-06, 0.00000000e+00, 0.00000000e+00,
       2.49937162e-05, 2.12389005e-05, 6.65320913e-07, 1.38577235e-07,
       2.99209619e-08, 0.00000000e+00, 0.00000000e+00, 7.03834398e-06,
       1.98153646e-08, 2.10254357e-07, 1.43823434e-06, 9.19062559e-03,
      